In [3]:
#####Path-Based Relevance from a Probabilistic Perspective#####
# ライブラリの読み込み
import numpy as np
import pandas as pd
import matplotlib.pyplot  as plt
import numpy.matlib
import scipy.linalg
import itertools
import scipy
from scipy import stats
from scipy import special
from scipy import sparse
from pandas.tools.plotting import scatter_matrix
from numpy.random import *

#np.random.seed(98537)

In [4]:

edges = 15
k = 10

theta = np.random.dirichlet(np.repeat(0.1, k), 1).reshape(-1)
phi = np.random.dirichlet(np.repeat(0.1, edges), k)


In [17]:
np.sum(theta * phi.T, axis=1)

array([1.59724227e-01, 9.64455756e-02, 1.09712052e-02, 2.43464893e-02,
       1.83226560e-01, 2.16942864e-01, 2.99349806e-04, 2.21290282e-01,
       2.08852115e-04, 1.48043863e-04, 3.25456994e-03, 5.61125217e-02,
       1.20196520e-02, 1.44685356e-02, 5.41272220e-04])

In [10]:
theta.reshape(1, k) * phi

ValueError: operands could not be broadcast together with shapes (1,10) (10,15) 